import numpy as np
import pandas as pd

df.head()
df.shape
df.info()

df.isnull().sum()
df.describe()

df=df.drop(['stn_code','sampling_date','agency','location_monitoring_station'],axis=1)
df=df.dropna(subset=['date'])

df.columns

df["type"].unique()

types = {
    "Residential" : "R" ,
    "Residential and others" : "RO" ,
    "Residential, Rural and other Areas" : "RRO",
    "Industrial Area" : "I",
    "Industrial Areas" : "I",
    "Industrial" : "I",
    "Sensitive Area" : "S",
    "Sensitive Areas" : "S",
    "Sensitive" : "S",
    "NaN" : "RRO"
}

df.type = df.type.replace(types)


df.head()

df['date'] = pd.to_datetime(df['date'], errors='coerce')
df.head(50)

df['year'] = df.date.dt.year
df.head(10)

COLS = ['so2', 'no2', 'rspm', 'spm', 'pm2_5']

import numpy as np
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
df[COLS] = imputer.fit_transform(df[COLS])

df['type'].value_counts()

df['type'].replace({"RRO":1,"I":2,"RO":3,"S":4,"RIRUO":5,"R":6},inplace=True)

df['state'].value_counts()

from sklearn.preprocessing import LabelEncoder
labelencoder=LabelEncoder()
df["state"]=labelencoder.fit_transform(df["state"])
df.head(60)

dfAndhra=df[(df['state']==0)]

dfAndhra['location'].value_counts()

from sklearn.preprocessing import OneHotEncoder
onehotencoder=OneHotEncoder(sparse=False,handle_unknown='error',drop='first')

pd.DataFrame(onehotencoder.fit_transform(dfAndhra[["location"]]))


import os
import matplotlib.pyplot as plt
import seaborn as sns

sns.boxplot(x=df['state'])
#No Outliers observed in 'state'

def remove_outliers(column):
    Q1 = column.quantile(0.25)
    Q3 = column.quantile(0.75)
    IQR = Q3-Q1
    threshold = 1.5 * IQR
    outlier_mask = (column < Q1 - threshold) | (column > Q3 + threshold)
    return column[~outlier_mask]
    
   
#Remove outliers for each column using a loop
col_name = ['so2', 'no2', 'rspm', 'spm', 'pm2_5']
for col in col_name:
    df[col] = remove_outliers(df[col])
    
    
import matplotlib.pyplot as ply
import seaborn as sns

plt.figure(figsize=(5, 5)) 

for col in col_name:
    sns.boxplot(data=df[col])
    ply.title(col)
    ply.show()
    
    
#Find the InterQuartile Range
Q1 = df.quantile(0.25)
Q3 = df.quantile(0.75)
IQR = Q3-Q1
print('*********** InterQuartile Range ***********')
print(IQR)
#Remove the outliers using IQR
df2 = df[~((df<(Q1-1.5*IQR))|(df>(Q3+1.5*IQR))).any(axis=1)]
df2.shape  


#Removing outliers using Z-score
from scipy import stats
z = np.abs(stats.zscore(df))
df3 = df[(z<3).all(axis=1)]
df3.shape